In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, mean_absolute_percentage_error, r2_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
from lightgbm import plot_importance
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import copy
import joblib
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler

In [310]:
data = pd.read_excel('data.xlsx')
data=data.reset_index()
data.columns = data.columns.str.replace(' ', '_')
data=data.drop('index',axis=1)

In [389]:
data_x=data.iloc[:,1:291]
data_y=data.iloc[:,0]
data_g=pd.DataFrame()

In [390]:
def remove_highly_correlated_features(data, threshold=0.9):
    corr_matrix = data.corr().abs() 
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))  
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]  #
    data_reduced = data.drop(columns=to_drop)
    return data_reduced, to_drop

data_x, dropped_features = remove_highly_correlated_features(data_x)

In [391]:
data_y=data.iloc[:,329]
data_X_train,data_X_test, data_y_train, data_y_test =train_test_split(data_x,data_y,test_size=0.2, random_state=23)

xgb_best_model = xgb.XGBRegressor(random_state=23)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

R-squared Score (R2): 0.7978


In [392]:

def xgb_cv(n_estimators, max_depth, learning_rate,):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'learning_rate': learning_rate
    }
    xgb_model = xgb.XGBRegressor(**params)
    
    cv_scores = cross_val_score(xgb_model, data_X_train, data_y_train, scoring='neg_root_mean_squared_error', cv=5)
    return cv_scores.mean()  

param_bounds = {
    'n_estimators': (100, 1000),
    'max_depth': (2, 7),
    'learning_rate': (0.01, 0.3)
}


optimizer = BayesianOptimization(
    f=xgb_cv,
    pbounds=param_bounds,
    verbose=2
)
optimizer.maximize(init_points=3, n_iter=7)

print("Best Parameters Found:", optimizer.max)


best_params = optimizer.max['params']
best_params['n_estimators'] = int(best_params['n_estimators'])  
best_params['max_depth'] = int(best_params['max_depth'])  

xgb_best_model = xgb.XGBRegressor(**best_params)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | -2.972e+0 | 0.2625    | 5.358     | 739.9     |
| 2         | -3.306e+0 | 0.2288    | 3.615     | 126.4     |
| 3         | -3.427e+0 | 0.2046    | 2.102     | 992.6     |
| 4         | -3.094e+0 | 0.2405    | 4.818     | 738.9     |
| 5         | -2.955e+0 | 0.04549   | 5.807     | 742.0     |
| 6         | -3.301e+0 | 0.05908   | 2.21      | 742.8     |
| 7         | -3.039e+0 | 0.05946   | 6.963     | 741.1     |
| 8         | -3.01e+03 | 0.2649    | 6.887     | 745.2     |
| 9         | -2.979e+0 | 0.06153   | 6.646     | 749.0     |
| 10        | -3.263e+0 | 0.3       | 3.767     | 749.5     |
Best Parameters Found: {'target': -2955.281006644144, 'params': {'learning_rate': 0.04549113363046612, 'max_depth': 5.807219904494478, 'n_estimators': 742.0415985737932}}
R-squared Score (R2): 0.6853


In [393]:
data_g['BTUNGPLHEAT']=xgb_best_model.predict(data_x)

In [394]:
data_y_i=data.iloc[:,330]
data_X_train,data_X_test, data_y_train, data_y_test =train_test_split(data_x,data_y_i,test_size=0.2, random_state=23)

xgb_best_model = xgb.XGBRegressor(random_state=23)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

R-squared Score (R2): 0.5079


In [395]:
def xgb_cv(n_estimators, max_depth, learning_rate,):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'learning_rate': learning_rate
    }
    xgb_model = xgb.XGBRegressor(**params)
    

    cv_scores = cross_val_score(xgb_model, data_X_train, data_y_train, scoring='neg_root_mean_squared_error', cv=5)
    return cv_scores.mean()  


param_bounds = {
    'n_estimators': (100, 1000),
    'max_depth': (2, 7),
    'learning_rate': (0.01, 0.3)
}


optimizer = BayesianOptimization(
    f=xgb_cv,
    pbounds=param_bounds,
    verbose=2
)
optimizer.maximize(init_points=3, n_iter=7)


print("Best Parameters Found:", optimizer.max)


best_params = optimizer.max['params']
best_params['n_estimators'] = int(best_params['n_estimators'])  
best_params['max_depth'] = int(best_params['max_depth'])  


xgb_best_model = xgb.XGBRegressor(**best_params)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | -2.599e+0 | 0.1979    | 6.668     | 866.0     |
| 2         | -2.57e+03 | 0.02835   | 5.401     | 182.4     |
| 3         | -2.511e+0 | 0.06348   | 3.965     | 429.7     |
| 4         | -2.51e+03 | 0.102     | 4.861     | 430.4     |
| 5         | -2.679e+0 | 0.3       | 7.0       | 487.3     |
| 6         | -2.679e+0 | 0.3       | 7.0       | 391.3     |
| 7         | -2.514e+0 | 0.07505   | 2.032     | 448.1     |
| 8         | -2.557e+0 | 0.2984    | 3.221     | 121.2     |
| 9         | -2.543e+0 | 0.07679   | 2.0       | 244.9     |
| 10        | -2.582e+0 | 0.2048    | 5.881     | 283.4     |
Best Parameters Found: {'target': -2509.528555965514, 'params': {'learning_rate': 0.10195902289505891, 'max_depth': 4.860886057245166, 'n_estimators': 430.40314160291234}}
R-squared Score (R2): 0.5479


In [396]:
data_g['KWHOTH']=xgb_best_model.predict(data_x)

In [397]:
data_y_i=data.iloc[:,331]
data_X_train,data_X_test, data_y_train, data_y_test =train_test_split(data_x,data_y_i,test_size=0.2, random_state=23)

xgb_best_model = xgb.XGBRegressor(random_state=23)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

R-squared Score (R2): 0.6001


In [398]:
def xgb_cv(n_estimators, max_depth, learning_rate,):
    params = {
        'n_estimators': int(n_estimators),
        'max_depth': int(max_depth),
        'learning_rate': learning_rate
    }
    xgb_model = xgb.XGBRegressor(**params)
    

    cv_scores = cross_val_score(xgb_model, data_X_train, data_y_train, scoring='neg_root_mean_squared_error', cv=5)
    return cv_scores.mean()  


param_bounds = {
    'n_estimators': (100, 1000),
    'max_depth': (2, 7),
    'learning_rate': (0.01, 0.3)
}

optimizer = BayesianOptimization(
    f=xgb_cv,
    pbounds=param_bounds,
    verbose=2
)
optimizer.maximize(init_points=3, n_iter=7)


print("Best Parameters Found:", optimizer.max)


best_params = optimizer.max['params']
best_params['n_estimators'] = int(best_params['n_estimators'])  
best_params['max_depth'] = int(best_params['max_depth'])  


xgb_best_model = xgb.XGBRegressor(**best_params)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
| 1         | -1.765e+0 | 0.1484    | 6.383     | 411.8     |
| 2         | -1.797e+0 | 0.193     | 5.502     | 992.0     |
| 3         | -1.817e+0 | 0.2722    | 6.989     | 499.4     |
| 4         | -1.845e+0 | 0.2915    | 6.041     | 413.0     |
| 5         | -1.756e+0 | 0.1108    | 5.411     | 992.0     |
| 6         | -1.818e+0 | 0.02814   | 3.281     | 318.6     |
| 7         | -1.737e+0 | 0.03622   | 6.041     | 411.7     |
| 8         | -1.825e+0 | 0.2843    | 6.347     | 793.6     |
| 9         | -1.812e+0 | 0.2771    | 3.947     | 426.5     |
| 10        | -1.737e+0 | 0.07556   | 5.325     | 339.3     |
Best Parameters Found: {'target': -1736.725753373334, 'params': {'learning_rate': 0.036221151069522325, 'max_depth': 6.0409321817827655, 'n_estimators': 411.69504625858616}}
R-squared Score (R2): 0.6464


In [399]:
data_g['KWHSPH']=xgb_best_model.predict(data_x)

In [400]:
data_y_i=data.iloc[:,332]
data_X_train,data_X_test, data_y_train, data_y_test =train_test_split(data_x,data_y_i,test_size=0.2, random_state=23)

xgb_best_model = xgb.XGBRegressor(random_state=23)
xgb_best_model.fit(data_X_train, data_y_train)

y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

R-squared Score (R2): 0.5916


In [401]:
data_g['KWHCOL']=xgb_best_model.predict(data_x)

In [402]:
data_y_i=data.iloc[:,333]
data_X_train,data_X_test, data_y_train, data_y_test =train_test_split(data_x,data_y_i,test_size=0.2, random_state=23)

xgb_best_model = xgb.XGBRegressor(random_state=23)
xgb_best_model.fit(data_X_train, data_y_train)


y_pred = xgb_best_model.predict(data_X_test)
r2 = r2_score(data_y_test, y_pred)
print("R-squared Score (R2): {:.4f}".format(r2))

R-squared Score (R2): 0.6621


In [403]:
data_g['KWHWTH']=xgb_best_model.predict(data_x)

In [404]:
data_x=data.iloc[:,1:291]
data_y=data.iloc[:,0]
data_X_train,data_X_test, data_y_train, data_y_test =train_test_split(data_x,data_y,test_size=0.2, random_state=23)

In [405]:
rf = RandomForestRegressor(max_depth=3,n_estimators=50, random_state=23)
rf.fit(data_X_train, data_y_train)

feature_importance = rf.feature_importances_
feature_names = data_x.columns


importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

top_features = importance_df.head(5)['Feature'].tolist()
data_select=data_x[top_features]
data_g=data_g.merge(data_select, how='inner', left_index=True, right_index=True)

In [406]:
xgb_best_model = xgb.XGBRegressor()
xgb_best_model.fit(data_X_train, data_y_train)

feature_importance = xgb_best_model.feature_importances_
feature_names = data_x.columns


importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
})
importance_df = importance_df.sort_values(by='Importance', ascending=False)


top_features = importance_df.head(5)['Feature'].tolist()
data_select=data_x[top_features]
data_g=data_g.merge(data_select, how='inner', left_index=True, right_index=True)

In [407]:
data_g['kwh']=data_y

In [408]:
data_g.to_csv('data_g.csv')